In [ ]:
import torch
from sentence_transformers import SentenceTransformer
model2 = SentenceTransformer('all-MiniLM-L6-v2')
state_dict = torch.load("BERT_STS_1C.pth")

# Set the loaded state dictionary to the model
model2.load_state_dict(state_dict)

In [ ]:
import csv


# Initialize an empty dictionary
test_data_dict = {}
test_data_dict["Sentence_1"]=[]
test_data_dict["Sentence_2"]=[]

# Open the CSV file
with open('sample_test.csv', mode='r') as file:
    # Create a CSV reader object
    reader = csv.reader(file)
    reader=list(reader)
    # Iterate over each row in the CSV file
    for i in range(1,len(reader)):
        row=reader[i]
        if len(row)==1:
            row=row[0].split('\t')
        else:
            row=''.join(row).split('\t')
        #print(row)
        test_data_dict["Sentence_1"].append(row[1])
        test_data_dict["Sentence_2"].append(row[2])

import torch
from transformers import BertTokenizer, BertForSequenceClassification
from scipy.stats import pearsonr
from torch.nn import CosineSimilarity
predicted_scores = []
with torch.no_grad():
      texts_a, texts_b = test_data_dict["Sentence_1"], test_data_dict["Sentence_2"]
      embeddings_a = model2.encode(texts_a, convert_to_tensor=True)
      embeddings_b = model2.encode(texts_b, convert_to_tensor=True)
      cosine_similarity = CosineSimilarity(dim=1)
      predicted_scores.extend(cosine_similarity(embeddings_a,embeddings_b))

predicted_scores = [float(value.to("cpu"))*5 for value in predicted_scores]
test_data_dict["Scores"] = predicted_scores

In [ ]:
pd.DataFrame(test_data_dict).to_csv("sample_demo.csv",sep = "\t")